In [12]:
import simpy
import numpy as np
import random

import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
import yaml
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import threading
import time


In [13]:
# === Config Parsing ===
def load_config(filename="config.yaml"):
    with open(filename, "r") as f:
        return yaml.safe_load(f)

In [14]:
# Load config from YAML and create expected CONFIG structure
raw_config = load_config()
print("Raw config loaded:", raw_config)

# Create CONFIG structure expected by the code
CONFIG = {
    'RAW_MATERIALS': raw_config['target_bottles'],
    'BUFFER_SIZES': {
        'generic': 20,  # For conveyor buffers
        'blow': raw_config['components']['blow']['capacity'],
        'buffer': 10,   # For intermediate buffers
        'clean': raw_config['components']['clean']['capacity'],
        'wrap': raw_config['components']['wrap']['capacity'],
        'storage': raw_config['components']['storage']['capacity']
    },
    'CONVEYOR_TIMES': {
        'conv_1': raw_config['components']['conv1']['speed_default'],
        'conv_2': raw_config['components']['conv2']['speed_default'],
        'conv_3': raw_config['components']['conv3']['speed_default'],
        'conv_4': raw_config['components']['conv4']['speed_default'],
        'conv_5': raw_config['components']['conv5']['speed_default'],
        'conv_6': raw_config['components']['conv6']['speed_default']
    },
    'CONVEYOR_INFO': {
        'min': 0.5,
        'max': 10.0,
        'fall_off_prob': 0.01
    },
    'MACHINE_INFO': {
        'min': 0.5,
        'max': 10.0
    },
    'SPEEDS': {
        'blow_molding': raw_config['components']['blow']['speed_default'],
        'cleaning': raw_config['components']['clean']['speed_default'],
        'wrapping': raw_config['components']['wrap']['speed_default'],
        'robotic_arm': raw_config['components']['robotic_arm']['speed_default']
    },
    'MIN_SPEED': 0.1,
    'MAX_SPEED': 5.0,
    'MIN_MACHINE_SPEED': 0.1,
    'MAX_MACHINE_SPEED': 5.0,
    'STEP_PER_FRAME': 10
}

print("\nCONFIG structure created:")
print(CONFIG)

Raw config loaded: {'target_bottles': 100, 'components': {'funnel': {'type': 'source', 'capacity': 200, 'speed': 100}, 'conv1': {'type': 'conveyor', 'capacity': 20, 'speed_min': 5, 'speed_max': 20, 'speed_default': 10}, 'blow': {'type': 'machine', 'capacity': 5, 'speed_min': 1, 'speed_max': 5, 'speed_default': 3}, 'conv2': {'type': 'conveyor', 'capacity': 20, 'speed_min': 5, 'speed_max': 20, 'speed_default': 10}, 'buffer1': {'type': 'buffer', 'capacity': 10}, 'conv3': {'type': 'conveyor', 'capacity': 10, 'speed_min': 5, 'speed_max': 20, 'speed_default': 10}, 'clean': {'type': 'machine', 'capacity': 3, 'speed_min': 1, 'speed_max': 5, 'speed_default': 2}, 'conv4': {'type': 'conveyor', 'capacity': 10, 'speed_min': 5, 'speed_max': 20, 'speed_default': 10}, 'buffer2': {'type': 'buffer', 'capacity': 5}, 'conv5': {'type': 'conveyor', 'capacity': 10, 'speed_min': 5, 'speed_max': 20, 'speed_default': 10}, 'wrap': {'type': 'machine', 'capacity': 2, 'speed_min': 1, 'speed_max': 3, 'speed_default'

In [ ]:
def sample_buffer_capacities(miu, sigma):
    # This is used to generate random environment configurations
    """
    Sample buffer capacities based on a normal distribution.
    Args:
        miu (list): List of mean of the normal distribution.
        sigma (list): List of standard deviation of the normal distribution.
    """
    capacities = []
    if len(miu) != len(sigma):
        raise ValueError("miu and sigma must have the same length")
    if type(miu) != list or type(sigma) != list:
        # no specific values provided, use default
        miu = [10, 10, 10, 50, 5, 50]
        sigma = [2, 2, 2, 5, 1, 10]
        for i in range(len(miu)):
            capacities.append(max(1, int(np.random.normal(miu[i], sigma[i]))))
    else:
        for i in range(len(miu)):
            capacities.append(max(1, int(np.random.normal(miu[i], sigma[i]))))
    return capacities

class Buffer:
    """Buffer class represents a buffer in the assembly line."""
    def __init__(self, env, name, capacity):
        self.env = env
        self.name = name
        self.capacity = capacity
        self.container = simpy.Store(env, capacity=capacity)
        self.level = 0

    def put(self, n=1):
        return self.container.put(n)

    def get(self, n=1):
        return self.container.get()

    def is_bottleneck(self):
        # if the buffer is full, it is a bottleneck
        return len(self.container.items) == self.capacity
    
    @property
    def items(self):
        """Get current items in buffer"""
        return self.container.items

class ConveyorBelt:
    """
    Enhanced ConveyorBelt with realistic backpressure handling, comprehensive statistics,
    and RL-friendly speed control for manufacturing simulation.
    
    Features:
    - Realistic backpressure: waits for downstream space before delivering items
    - Comprehensive statistics tracking (utilization, throughput, efficiency, etc.)
    - Random variability in transport times for realism
    - RL-controllable speed with reasonable bounds
    - Fall-off probability simulation for failure modes
    """
    def __init__(self, env, name, input_buffer, output_buffer, timing_config, base_time, fall_off_prob=0.01):
        self.env = env
        self.name = name
        self.input_buffer = input_buffer
        self.output_buffer = output_buffer
        self.capacity = 5  # Default capacity for items in transit

        if isinstance(timing_config, list) and len(timing_config) >= 2:
            # If timing_config is a list [min, max], use base_time as the base
            self.min_conveyor_time = timing_config[0]
            self.max_conveyor_time = timing_config[1]
            self.base_conveyor_time = base_time
        elif isinstance(timing_config, dict):
            # If timing_config is a dictionary, use its values
            self.base_conveyor_time = timing_config.get('base', base_time)
            self.min_conveyor_time = timing_config.get('min', base_time * 0.5)
            self.max_conveyor_time = timing_config.get('max', base_time * 2.0)
        else:
            # If it's a single value, use it for all times
            self.base_conveyor_time = base_time
            self.min_conveyor_time = base_time * 0.5
            self.max_conveyor_time = base_time * 2.0
        
        self.conveyor_time = self.base_conveyor_time  # Current conveyor time
        self.time_param = self.base_conveyor_time  # For RL compatibility
        self.items_in_transit = 0  # Count of items currently on the conveyor
        self.bottle_fall_off_prob = fall_off_prob  # Probability of bottle falling off the conveyor
        
        # Statistics tracking
        self.wait_times = []  # List to store the wait times
        self.transported_items = 0  # Total items successfully transported
        self.fall_off_count = 0  # Count of bottles that fall off the conveyor
        self.total_transport_time = 0  # Total time spent transporting items
        self.idle_time = 0  # Total time spent idle (waiting for items)
        self.blocked_time = 0  # Total time spent blocked by downstream
        self.last_idle_start = 0  # For tracking idle periods
        self.last_blocked_start = 0  # For tracking blocked periods
        
        # Conveyor state
        self.is_busy = False  # Whether conveyor is actively transporting
        self.is_blocked = False  # Whether conveyor is blocked by downstream
        self.speed_factor = 1.0  # For RL control (adjustable speed)
        
        # Start the conveyor process
        self.proc = env.process(self.run())

    def run(self):
        while True:
            # Record when we start waiting for an item (idle time tracking)
            self.last_idle_start = self.env.now
            
            # Wait for item from input buffer
            yield self.input_buffer.get()
            # Calculate and track wait time
            wait_time = self.env.now - self.last_idle_start
            self.wait_times.append(wait_time)
            self.idle_time += wait_time

            # Check conveyor capacity
            if self.items_in_transit >= self.capacity:
                # Track blocked time start
                self.last_blocked_start = self.env.now
                self.is_blocked = True

                # Wait for either:
                # 1. Item to leave conveyor (space available), OR
                # 2. Output buffer to have space
                while (self.items_in_transit >= self.capacity or 
                    len(self.output_buffer.container.items) >= self.output_buffer.capacity):
                    # Check if item falls off during wait
                    if random.random() < self.bottle_fall_off_prob:
                        self.fall_off_count += 1
                         # Track blocked time before breaking
                        blocked_time = self.env.now - self.last_blocked_start
                        self.blocked_time += blocked_time
                        self.is_blocked = False
                        break  # Item falls off, exit wait loop
                    yield self.env.timeout(0.01)  # Short polling interval
                else:
                    # Space became available
                    blocked_time = self.env.now - self.last_blocked_start
                    self.blocked_time += blocked_time
                    self.is_blocked = False
                    
                    # Item enters conveyor
                    self.items_in_transit += 1
                    self.is_busy = True
                    # Start transport process
                    self.env.process(self.transport_item())
            else:
                # Item enters conveyor directly
                self.items_in_transit += 1
                self.is_busy = True
                # Start transport process
                self.env.process(self.transport_item())

    def transport_item(self):
        """Handle individual item transport"""
        transport_start = self.env.now
        conveyor_time = max(self.min_conveyor_time, min(self.conveyor_time, self.max_conveyor_time))
        # TODO: add random variability to conveyor time
        yield self.env.timeout(conveyor_time)
        
        # Check for random failures during transport
        if random.random() < self.bottle_fall_off_prob:
            self.items_in_transit -= 1
            self.fall_off_count += 1
            self.is_busy = False
            return
        
        # Track blocked time if waiting for output buffer
        wait_start = self.env.now
        was_blocked = False
        
        # Wait for output buffer space (realistic backpressure)
        while len(self.output_buffer.container.items) >= self.output_buffer.capacity:
            if not was_blocked:
                was_blocked = True
                self.is_blocked = True
            
            if random.random() < self.bottle_fall_off_prob * 2:  # Higher chance when blocked
                self.items_in_transit -= 1
                self.fall_off_count += 1
                # Track blocked time
                if was_blocked:
                    self.blocked_time += self.env.now - wait_start
                    self.is_blocked = False
                self.is_busy = False
                return
            yield self.env.timeout(0.1)  # Wait for space
        
        # Track blocked time if we were blocked
        if was_blocked:
            self.blocked_time += self.env.now - wait_start
            self.is_blocked = False


        # Successfully deliver item
        yield self.output_buffer.put(1)
        self.items_in_transit -= 1
        self.transported_items += 1

        # Track total transport time
        total_time = self.env.now - transport_start
        self.total_transport_time += total_time
        
        self.is_busy = False

    def set_speed(self, speed_factor):
        """Set conveyor speed for RL control (1.0 = normal, >1.0 = faster)"""
        self.speed_factor = max(0.1, min(speed_factor, 3.0))  # Reasonable bounds
        
        # Update current conveyor time for display
        self.conveyor_time = self.base_conveyor_time / self.speed_factor

    def get_utilization(self):
        """Calculate conveyor utilization percentage"""
        if self.env.now == 0:
            return 0.0
        # Utilization = time spent actually transporting / total time
        return (self.total_transport_time / self.env.now) * 100

    def get_throughput(self):
        """Calculate items per unit time"""
        if self.env.now == 0:
            return 0.0
        return self.transported_items / self.env.now

    def get_efficiency(self):
        """Calculate efficiency (successful transports / total attempts)"""
        total_attempts = self.transported_items + self.fall_off_count
        if total_attempts == 0:
            return 100.0
        return (self.transported_items / total_attempts) * 100

    def get_avg_wait_time(self):
        """Calculate average wait time for items"""
        if not self.wait_times:
            return 0.0
        return sum(self.wait_times) / len(self.wait_times)

    def get_stats(self):
        """Return comprehensive conveyor statistics"""
        return {
            'name': self.name,
            'transported_items': self.transported_items,
            'fall_off_count': self.fall_off_count,
            'items_in_transit': self.items_in_transit,
            'utilization': self.get_utilization(),
            'throughput': self.get_throughput(),
            'efficiency': self.get_efficiency(),
            'avg_wait_time': self.get_avg_wait_time(),
            'total_idle_time': self.idle_time,
            'total_blocked_time': self.blocked_time,
            'is_busy': self.is_busy,
            'is_blocked': self.is_blocked,
            'current_speed_factor': self.speed_factor,
            'current_conveyor_time': self.conveyor_time
        }


class Machine:
    """
    Machine simulates a processing machine that takes items from an input buffer,
    processes them for a specified time, and puts them into an output buffer.
    It tracks wait times and idle times, and can adjust its processing speed.
    The processing speed can be adjusted within a defined range.
    TODO: increase process time when buffer is full, decrease when buffer is empty.
    TODO: if buffer is full for a long time, add machine fix time.
    TODO: add a wrapping machine that wraps multiple items at once.
    Args:
        env (simpy.Environment): Simulation environment.
        name (str): Name of the machine.
        input_buffer (Buffer): Buffer to get items from.
        output_buffer (Buffer): Buffer to put items into.
        speed (float): Processing speed of the machine.
    """
    def __init__(self, env, name, input_buffer, output_buffer, time_config, base_speed):
        self.env = env
        self.name = name
        self.input_buffer = input_buffer
        self.output_buffer = output_buffer
        
        if isinstance(time_config, list) and len(time_config) >= 2:
            # If time_config is a list [min, max], use base_speed as the base
            self.min_time = time_config[0]
            self.max_time = time_config[1]
            self.base_process_time = base_speed
        elif isinstance(time_config, dict):
            # If time_config is a dictionary
            self.base_process_time = time_config.get('base', base_speed)
            self.min_time = time_config.get('min', base_speed * 0.5)
            self.max_time = time_config.get('max', base_speed * 2.0)
        else:
            # Fallback to simple configuration
            self.base_process_time = base_speed
            self.min_time = base_speed * 0.5
            self.max_time = base_speed * 2.0

        # Initialize current_process_time for RL compatibility
        self.current_process_time = self.base_process_time
        self.speed = base_speed  # For RL compatibility

         # Statistics tracking
        self.wait_times = []
        self.processed_items = 0
        self.total_processing_time = 0
        self.idle_time = 0
        self.last_idle_start = 0
        
        # Machine state
        self.is_busy = False
        self.speed_factor = 1.0  # For RL control
        
        # Start the machine process
        self.proc = env.process(self.run())

    def run(self):
        while True:
            # Record when we start waiting for an item (idle time tracking)
            self.last_idle_start = self.env.now
            
            # Wait for an item to be available
            yield self.input_buffer.get()
            
            # Calculate wait time and update idle time
            wait_time = self.env.now - self.last_idle_start
            self.wait_times.append(wait_time)
            self.idle_time += wait_time
            
            # Mark machine as busy
            self.is_busy = True
            
            # Calculate actual processing time
            actual_process_time = self.get_processing_time()
            
            # Process the item
            yield self.env.timeout(actual_process_time)
            
            # Update statistics
            self.processed_items += 1
            self.total_processing_time += actual_process_time
            
            # Mark machine as not busy
            self.is_busy = False
            
            # Put item to output buffer (with backpressure handling)
            yield self.put_to_output()

    def get_processing_time(self):
        """Calculate actual processing time with speed factor and bounds"""
        # Apply speed factor (higher speed_factor = faster processing)
        adjusted_time = self.current_process_time / self.speed_factor
        
        # Add some realistic variability (±10%)
        variability = 0.1
        varied_time = adjusted_time * (1 + random.uniform(-variability, variability))
        
        # Clamp to realistic bounds
        return max(self.min_time, min(varied_time, self.max_time))
    
    def put_to_output(self):
        """Put item to output buffer with backpressure handling"""
        # Wait for space in output buffer if it's full
        while len(self.output_buffer.container.items) >= self.output_buffer.capacity:
            yield self.env.timeout(0.1)  # Wait for space
        
        # Put item into output buffer
        yield self.output_buffer.put(1)
    
    def set_speed(self, new_speed):
        """Set machine processing speed (for RL control)"""
        # Speed factor: 1.0 = normal, >1.0 = faster, <1.0 = slower
        self.speed_factor = max(0.1, min(new_speed, 5.0))  # Reasonable bounds
        
        # Update current process time for display/logging
        self.current_process_time = self.base_process_time / self.speed_factor
    
    def get_utilization(self):
        """Calculate machine utilization percentage"""
        if self.env.now == 0:
            return 0.0
        total_time = self.env.now
        return (self.total_processing_time / total_time) * 100
    
    def get_stats(self):
        """Return comprehensive machine statistics"""
        avg_wait = sum(self.wait_times) / len(self.wait_times) if self.wait_times else 0
        return {
            'name': self.name,
            'processed_items': self.processed_items,
            'utilization': self.get_utilization(),
            'avg_wait_time': avg_wait,
            'total_idle_time': self.idle_time,
            'is_busy': self.is_busy,
            'current_speed_factor': self.speed_factor,
            'current_process_time': self.current_process_time
        }

class RoboticArm:
    """
    RoboticArm simulates a robotic arm that transfers items from one buffer to another.
    Can pick how much items to transfer at once, and has a speed parameter that affects how long it takes to transfer items.
    TODO: add how many items to transfer at once. Larger -> slower, but moves more items at once
    Args:
        env (simpy.Environment): Simulation environment.
        input_buffer (Buffer): Buffer to get items from.
        output_buffer (Buffer): Buffer to put items into.
        speed (float): Speed of the robotic arm.
    """
    def __init__(self, env, input_buffer, output_buffer, speed_config, batch_size=1):
        self.env = env
        self.input_buffer = input_buffer
        self.output_buffer = output_buffer
        self.batch_size = batch_size
        
        # Configure timing
        if isinstance(speed_config, dict):
            self.base_cycle_time = speed_config.get('base', 2.0)
            self.min_cycle_time = speed_config.get('min', 0.5)
            self.max_cycle_time = speed_config.get('max', 10.0)
            self.pickup_time = speed_config.get('pickup', 0.5)
            self.transport_time = speed_config.get('transport', 1.0)
            self.drop_time = speed_config.get('drop', 0.3)
            self.batch_strategy = speed_config.get('batch_strategy', 'wait_for_full')
        else:
            # Simple float configuration
            self.base_cycle_time = speed_config
            self.min_cycle_time = speed_config * 0.3
            self.max_cycle_time = speed_config * 3.0
            self.pickup_time = speed_config * 0.25
            self.transport_time = speed_config * 0.5
            self.drop_time = speed_config * 0.15
            self.batch_strategy = 'wait_for_full'  # Default strategy
        
        # For RL compatibility
        self.speed = speed_config if isinstance(speed_config, (int, float)) else self.base_cycle_time
        
        # Statistics tracking
        self.transferred_items = 0
        self.total_cycles = 0
        self.total_cycle_time = 0
        self.wait_times = []
        self.idle_time = 0
        self.last_idle_start = 0
        self.blocked_time = 0
        self.last_blocked_start = 0
        
        # Batch statistics
        self.batch_sizes = []
        self.avg_batch_size = 0
        
        # Arm state
        self.is_busy = False
        self.is_blocked = False
        self.current_batch = 0
        self.speed_factor = 1.0  # For RL control
        
        # Failure simulation
        self.failure_prob = 0.001  # Very low chance of mechanical failure
        self.maintenance_needed = False
        
        # Start the robotic arm process
        self.proc = env.process(self.run())
    def run(self):
        while True:
            self.last_idle_start = self.env.now
            
            # Get batch based on strategy
            if self.batch_strategy == "wait_for_full":
                items = yield self.env.process(self.wait_for_full_batch())
            elif self.batch_strategy == "adaptive":
                items = yield self.env.process(self.adaptive_batch_collection())
            else:  # timeout strategy
                items = yield self.env.process(self.timeout_batch_collection())
            
            # Update statistics
            wait_time = self.env.now - self.last_idle_start
            self.wait_times.append(wait_time)
            self.idle_time += wait_time
            
            self.current_batch = len(items)
            self.batch_sizes.append(self.current_batch)
            self.is_busy = True
            
            # Process the batch
            cycle_start = self.env.now
            yield self.env.process(self.process_batch(items))
            
            # Update cycle statistics
            cycle_time = self.env.now - cycle_start
            self.total_cycles += 1
            self.total_cycle_time += cycle_time
            self.transferred_items += len(items)
            
            self.is_busy = False

    def wait_for_full_batch(self):
        """Wait until full batch size is available"""
        while len(self.input_buffer.container.items) < self.batch_size:
            yield self.env.timeout(0.01)  # Check every 0.1 time units
        
        # Collect full batch
        items = []
        for _ in range(self.batch_size):
            item = yield self.input_buffer.get()
            items.append(item)
        
        return items

    def adaptive_batch_collection(self):
        """Adaptive strategy: wait for full batch but accept partial if nothing new arrives"""
        items = []
        
        # First, wait for at least one item
        if len(self.input_buffer.container.items) == 0:
            item = yield self.input_buffer.get()
            items.append(item)
        
        # Then try to collect more for full batch
        patience_start = self.env.now
        last_seen_count = len(self.input_buffer.container.items)
        
        while len(items) < self.batch_size:
            current_available = len(self.input_buffer.container.items)
            
            if current_available > 0:
                # Take one more item
                item = yield self.input_buffer.get()
                items.append(item)
                last_seen_count = len(self.input_buffer.container.items)
                patience_start = self.env.now  # Reset patience timer
            else:
                # No items available, wait a bit
                yield self.env.timeout(0.2)
                
                # Check if we've been patient enough
                if self.env.now - patience_start > 1.0:  # 1 time unit of patience
                    break  # Accept partial batch
        
        return items

    def timeout_batch_collection(self):
        """Timeout strategy: wait for full batch but timeout after max wait"""
        start_time = self.env.now
        items = []
        
        # Wait for at least one item
        if len(self.input_buffer.container.items) == 0:
            item = yield self.input_buffer.get()
            items.append(item)
        
        # Try to collect full batch within timeout
        while (len(items) < self.batch_size and 
               self.env.now - start_time < self.batch_timeout):
            
            if len(self.input_buffer.container.items) > 0:
                item = yield self.input_buffer.get()
                items.append(item)
            else:
                yield self.env.timeout(0.1)  # Wait a bit for more items
        
        return items

    def set_batch_strategy(self, strategy, timeout=None, min_batch=None):
        """Change batching strategy during simulation (for RL control)"""
        self.batch_strategy = strategy
        if timeout is not None:
            self.batch_timeout = timeout
        if min_batch is not None:
            self.min_batch_size = min_batch

    def process_batch(self, items):
        """Process a batch of items through pickup, transport, and drop phases"""
        batch_size = len(items)
        
        # Check for mechanical failure
        if random.random() < self.failure_prob:
            self.maintenance_needed = True
            yield self.env.timeout(10.0)  # Maintenance time
            self.maintenance_needed = False
        
        # Phase 1: Pickup time (increases slightly with batch size)
        pickup_time = self.pickup_time * (1 + 0.1 * (batch_size - 1))
        pickup_time = pickup_time / self.speed_factor
        pickup_time = max(self.min_cycle_time * 0.1, pickup_time)
        yield self.env.timeout(pickup_time)
        
        # Phase 2: Transport time (base time regardless of batch size)
        transport_time = self.transport_time / self.speed_factor
        transport_time = max(self.min_cycle_time * 0.3, transport_time)
        yield self.env.timeout(transport_time)
        
        # Phase 3: Drop items (no backpressure handling needed)
        for item in items:
            # Drop time per item (faster for batch operations)
            drop_time_per_item = self.drop_time / self.speed_factor
            if batch_size > 1:
                drop_time_per_item *= 0.8  # 20% faster per item in batch
            
            yield self.env.timeout(drop_time_per_item)
            yield self.output_buffer.put(item)  # No waiting needed

    def set_speed(self, speed_factor):
        """Set robotic arm speed for RL control (1.0 = normal, >1.0 = faster)"""
        self.speed_factor = max(0.2, min(speed_factor, 3.0))  # Reasonable bounds for robotic arm

    def set_batch_size(self, new_batch_size):
        """Dynamically adjust batch size (for RL control)"""
        self.batch_size = max(1, min(new_batch_size, 10))  # Reasonable bounds

    def get_utilization(self):
        """Calculate arm utilization percentage"""
        if self.env.now == 0:
            return 0.0
        return (self.total_cycle_time / self.env.now) * 100

    def get_throughput(self):
        """Calculate items per unit time"""
        if self.env.now == 0:
            return 0.0
        return self.transferred_items / self.env.now

    def get_avg_wait_time(self):
        """Calculate average wait time"""
        if not self.wait_times:
            return 0.0
        return sum(self.wait_times) / len(self.wait_times)

    def get_avg_batch_size(self):
        """Calculate average batch size"""
        if not self.batch_sizes:
            return 0.0
        return sum(self.batch_sizes) / len(self.batch_sizes)

    def get_cycle_efficiency(self):
        """Calculate how efficiently batches are being used"""
        if not self.batch_sizes:
            return 100.0
        avg_batch = self.get_avg_batch_size()
        return (avg_batch / self.batch_size) * 100

    def get_stats(self):
        """Return comprehensive robotic arm statistics"""
        return {
            'name': self.name,
            'transferred_items': self.transferred_items,
            'total_cycles': self.total_cycles,
            'utilization': self.get_utilization(),
            'throughput': self.get_throughput(),
            'avg_wait_time': self.get_avg_wait_time(),
            'avg_batch_size': self.get_avg_batch_size(),
            'cycle_efficiency': self.get_cycle_efficiency(),
            'total_idle_time': self.idle_time,
            'total_blocked_time': self.blocked_time,
            'is_busy': self.is_busy,
            'is_blocked': self.is_blocked,
            'current_batch_size': self.current_batch,
            'max_batch_size': self.batch_size,
            'current_speed_factor': self.speed_factor,
            'maintenance_needed': self.maintenance_needed
        }

    def set_speed(self, speed_factor):
        """Set robotic arm speed for RL control (1.0 = normal, >1.0 = faster)"""
        self.speed_factor = max(0.2, min(speed_factor, 3.0))  # Reasonable bounds for robotic arm
        # Update speed attribute for compatibility
        self.speed = self.base_cycle_time / self.speed_factor
    

def raw_material_source(env, count, output_buffer):
    for _ in range(count):
        yield output_buffer.put(1)

def setup_simulation(env, speeds=None, buffer_sizes=None, conveyor_times=None):
    bs = buffer_sizes or CONFIG['BUFFER_SIZES']
    ct = conveyor_times or [
        CONFIG['CONVEYOR_TIMES']['conv_1'],
        CONFIG['CONVEYOR_TIMES']['conv_2'],
        CONFIG['CONVEYOR_TIMES']['conv_3'],
        CONFIG['CONVEYOR_TIMES']['conv_4'],
        CONFIG['CONVEYOR_TIMES']['conv_5'],
        CONFIG['CONVEYOR_TIMES']['conv_6'],
    ]
    conv_times = [CONFIG['CONVEYOR_INFO']['min'], CONFIG['CONVEYOR_INFO']['max']]
    conv_fall_off_prob = CONFIG['CONVEYOR_INFO']['fall_off_prob']
    
    machine_times = [CONFIG['MACHINE_INFO']['min'], CONFIG['MACHINE_INFO']['max']]

    buffers = {
        'funnel': Buffer(env, CONFIG['RAW_MATERIALS'], 'Funnel'),
        'conveyor_1': Buffer(env, bs['generic'], 'Conveyor 1'),
        'blow': Buffer(env, bs['blow'], 'Blow Buffer'),
        'conveyor_2': Buffer(env, bs['generic'], 'Conveyor 2'),
        'buffer_1': Buffer(env, bs['buffer'], 'Buffer 1'),
        'conveyor_3': Buffer(env, bs['generic'], 'Conveyor 3'),
        'clean': Buffer(env, bs['clean'], 'Clean Buffer'),
        'conveyor_4': Buffer(env, bs['generic'], 'Conveyor 4'),
        'buffer_2': Buffer(env, bs['buffer'], 'Buffer 2'),
        'conveyor_5': Buffer(env, bs['generic'], 'Conveyor 5'),
        'wrap': Buffer(env, bs['wrap'], 'Wrap Buffer'),
        'conveyor_6': Buffer(env, bs['generic'], 'Conveyor 6'),
        'buffer_3': Buffer(env, bs['buffer'], 'Buffer 3'),
        'storage': Buffer(env, bs['storage'], 'Storage Buffer'),
        'platform': Buffer(env, CONFIG['RAW_MATERIALS'], 'Storage Platform'),
    }

    env.process(raw_material_source(env, CONFIG['RAW_MATERIALS'], buffers['funnel']))

    conveyors = []
    conveyors.append(ConveyorBelt(env, "Funnel to Conveyor 1", buffers['funnel'], buffers['conveyor_1'],conv_times, ct[0],conv_fall_off_prob))
    conveyors.append(ConveyorBelt(env, "Conveyor 1 to Blow", buffers['conveyor_1'], buffers['blow'],conv_times, ct[0],conv_fall_off_prob))
    m1 = Machine(env, "Blow Molding", buffers['blow'], buffers['conveyor_2'], machine_times, speeds[0] if speeds is not None else CONFIG['SPEEDS']['blow_molding'])
    conveyors.append(ConveyorBelt(env, "Conveyor 2 to Buffer 1", buffers['conveyor_2'], buffers['buffer_1'],conv_times, ct[1],conv_fall_off_prob))
    conveyors.append(ConveyorBelt(env, "Buffer 1 to Conveyor 3", buffers['buffer_1'], buffers['conveyor_3'],conv_times, ct[2],conv_fall_off_prob))
    conveyors.append(ConveyorBelt(env, "Conveyor 3 to Clean", buffers['conveyor_3'], buffers['clean'], conv_times,ct[2],conv_fall_off_prob))
    m2 = Machine(env, "Cleaning/Filling", buffers['clean'], buffers['conveyor_4'], machine_times, speeds[1] if speeds is not None else CONFIG['SPEEDS']['cleaning'])
    conveyors.append(ConveyorBelt(env, "Conveyor 4 to Buffer 2", buffers['conveyor_4'], buffers['buffer_2'],conv_times, ct[3],conv_fall_off_prob))
    conveyors.append(ConveyorBelt(env, "Buffer 2 to Conveyor 5", buffers['buffer_2'], buffers['conveyor_5'],conv_times, ct[4],conv_fall_off_prob))
    conveyors.append(ConveyorBelt(env, "Conveyor 5 to Wrap", buffers['conveyor_5'], buffers['wrap'], conv_times,ct[4],conv_fall_off_prob))
    m3 = Machine(env, "Wrapping", buffers['wrap'], buffers['conveyor_6'], machine_times, speeds[2] if speeds is not None else CONFIG['SPEEDS']['wrapping'])
    conveyors.append(ConveyorBelt(env, "Conveyor 6 to Buffer 3", buffers['conveyor_6'], buffers['buffer_3'],conv_times, ct[5],conv_fall_off_prob))
    conveyors.append(ConveyorBelt(env, "Buffer 3 to Storage", buffers['buffer_3'], buffers['storage'],conv_times, ct[5],conv_fall_off_prob))
    robotic_arm = RoboticArm(env, buffers['storage'], buffers['platform'], speeds[3] if speeds is not None else CONFIG['SPEEDS']['robotic_arm'])

    machines = [m1, m2, m3, robotic_arm]
    return buffers, machines, conveyors

# --- RL-READY ENVIRONMENT ---
class AssemblyLineEnv:
    def __init__(self, config=CONFIG):
        self.config = config
        self.env = None
        self.buffers = None
        self.machines = None
        self.conveyors = None
        self.n_machines = 4
        self.n_conveyors = 6  # Update if you change the number above
        self.current_speeds = [
            config['SPEEDS']['blow_molding'],
            config['SPEEDS']['cleaning'],
            config['SPEEDS']['wrapping'],
            config['SPEEDS']['robotic_arm'],
        ]
        self.current_conveyor_times = [
            config['CONVEYOR_TIMES']['conv_1'],
            config['CONVEYOR_TIMES']['conv_2'],
            config['CONVEYOR_TIMES']['conv_3'],
            config['CONVEYOR_TIMES']['conv_4'],
            config['CONVEYOR_TIMES']['conv_5'],
            config['CONVEYOR_TIMES']['conv_6'],
        ]
        self.sim_steps = 0
        self.throughput = 0.0
        self.prev_platform_count = 0

    def reset(self):
        buffer_sizes = None
        self.env = simpy.Environment()
        self.buffers, self.machines, self.conveyors = setup_simulation(
            self.env, self.current_speeds, buffer_sizes, self.current_conveyor_times
        )
        self.done = False
        self.sim_steps = 0
        self.throughput = 0.0
        self.prev_platform_count = 0
        return self._get_obs()

    def step(self, action):
        # Split action into machine speeds and conveyor times
        action[:4] = np.clip(action[:4], self.config['MIN_MACHINE_SPEED'], self.config['MAX_MACHINE_SPEED'])
        action[4:] = np.clip(action[4:], self.config['MIN_SPEED'], self.config['MAX_SPEED'])
        machine_speeds = action[:self.n_machines]
        if hasattr(self.machines[i], 'set_speed'):
            self.machines[i].set_speed(float(new_speed))
        else:
            self.machines[i].speed = float(new_speed)
        for i, new_time in enumerate(conveyor_times):
            if i < len(self.conveyors):
                if hasattr(self.conveyors[i], 'set_speed'):
                    self.conveyors[i].set_speed(float(new_time))
                else:
                    self.conveyors[i].time_param = float(new_time)
        steps = self.config['STEP_PER_FRAME']
        for _ in range(steps):
            if len(self.buffers['platform'].items) < self.config['RAW_MATERIALS']:
                self.env.step()
        self.sim_steps += steps
        obs = self._get_obs()
        # Throughput: items on platform / total sim time
        current_platform_count = len(self.buffers['platform'].items)
        if self.sim_steps > 0:
            self.throughput = current_platform_count / self.sim_steps
        reward = self._get_reward()
        self.done = (current_platform_count >= self.config['RAW_MATERIALS'])
        return obs, reward, self.done, {}
    
    def step_validation(self, action):
        # Split action into machine speeds and conveyor times
        action = np.clip(action, self.config['MIN_SPEED'], self.config['MAX_SPEED'])
        machine_speeds = action[:self.n_machines]
        conveyor_times = action[self.n_machines:self.n_machines + self.n_conveyors]
        for i, new_speed in enumerate(machine_speeds):
            if i < len(self.machines):
                if hasattr(self.machines[i], 'set_speed'):
                    self.machines[i].set_speed(float(new_speed))
                else:
                    self.machines[i].speed = float(new_speed)
        for i, new_time in enumerate(conveyor_times):
            if i < len(self.conveyors):
                if hasattr(self.conveyors[i], 'set_speed'):
                    self.conveyors[i].set_speed(float(new_time))
                else:
                    self.conveyors[i].time_param = float(new_time)
        steps = 50
        for _ in range(steps):
            if len(self.buffers['platform'].items) < self.config['RAW_MATERIALS']:
                self.env.step()
                full_buffers = sum(
                    len(self.buffers[name].items) >= self.buffers[name].capacity
                    for name in self.buffers if self.buffers[name].capacity > 0 and name != 'platform' and name != 'funnel'
                )
                if full_buffers > 0:
                    print(f"Warning: {full_buffers} buffers are full, consider adjusting speeds or conveyor times.")
        self.sim_steps += steps
        obs = self._get_obs()
        # Throughput: items on platform / total sim time
        current_platform_count = len(self.buffers['platform'].items)
        if self.sim_steps > 0:
            self.throughput = current_platform_count / self.sim_steps
        reward = self._get_reward()
        self.done = (current_platform_count >= self.config['RAW_MATERIALS'])
        return obs, reward, self.done, {}
            len(self.buffers[name].items) / self.buffers[name].capacity
    def _get_obs(self):
        return np.array([
            len(self.buffers[name].items) / self.buffers[name].capacity
            for name in self.buffers
        ], dtype=np.float32)

    def _get_reward(self):
        full_buffers = sum(
            len(self.buffers[name].items) >= self.buffers[name].capacity
            for name in self.buffers if self.buffers[name].capacity > 0 and name != 'platform' and name != 'funnel'
        )
        return -full_buffers + 0.01 * len(self.buffers['platform'].items)

    def render(self):
        print(" | ".join(f"{name}:{len(self.buffers[name].items)}" for name in self.buffers))

# --- GYM WRAPPER ---
class AssemblyLineGymEnv(gym.Env):
    def __init__(self, config=None):
        super().__init__()
        self.backend = AssemblyLineEnv(config or CONFIG)
        self.backend.reset()
        # 4 machines + 11 conveyors
        self.action_space = spaces.Box(
            low=np.array([CONFIG['MIN_SPEED']] * 4 + [2.0] * 11),
            high=np.array([CONFIG['MAX_SPEED']] * 4 + [10.0] * 11),
            dtype=np.float32
        )
        obs_size = len(self.backend.buffers)
        self.observation_space = spaces.Box(
            low=0.0, high=1.0, shape=(obs_size,), dtype=np.float32
        )

    def reset(self, *, seed=None, options=None):
        obs = self.backend.reset()
        return obs, {}

    def step(self, action):
        obs, reward, done, info = self.backend.step(action)
        return obs, reward, done, False, info
    
    def step_validation(self, action):
        obs, reward, done, info = self.backend.step_validation(action)
        return obs, reward, done, False, info

    def render(self):
        self.backend.render()

    def close(self):
        pass
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def animate_validation(agent, env, config=CONFIG, save_path=None, show=True):
    obs, _ = env.reset()
    done = False

    buffer_names = list(env.backend.buffers.keys())
    buffer_caps = [env.backend.buffers[n].capacity for n in buffer_names]
    buffer_history = []
        action, _ = agent.predict(obs, deterministic=True)
    # Collect history as we step through the episode
    while not done:
        action, _ = agent.predict(obs, deterministic=True)
        obs, reward, done, _, _ = env.step_validation(action)
        buffer_history.append([len(env.backend.buffers[n].items) for n in buffer_names])
    buffer_history = buffer_history[::skip]
    buffer_history = np.array(buffer_history)
    skip = 1 # Change this to control how many frames you skip
    buffer_history = buffer_history[::skip]
    n_steps = buffer_history.shape[0]
    # n_steps = buffer_history.shape[0]

    fig, ax = plt.subplots(figsize=(14, 6))
    bar_container = ax.bar(buffer_names, buffer_history[0], color="skyblue")

    def update(frame):
        heights = buffer_history[frame]
        colors = [
            "red" if heights[i] >= buffer_caps[i] else "skyblue"
            for i in range(len(buffer_names))
        ]
        for rect, h, color in zip(bar_container, heights, colors):
            rect.set_height(h)
            rect.set_color(color)
        ax.set_ylim(0, max(buffer_caps + [10]))
        ax.set_title(f"Sim time: {frame * config['STEP_PER_FRAME']}")
        return bar_container

    plt.xticks(rotation=60, ha='right')
    plt.tight_layout()
    ani = FuncAnimation(
        fig, update, frames=n_steps,
        repeat=False, blit=False, interval=100
    )
    if save_path:
        print(f"Saving animation to {save_path}...")
        print("total frames:", n_steps)
        ani.save(save_path, writer='ffmpeg', fps=10)
    if show:
        plt.show()
    plt.close(fig)

def linear_schedule(progress_remaining):
    """Linearly decrease learning rate from 3e-4 to 1e-5."""
    return 1e-5 + (3e-4 - 1e-5) * progress_remaining
from torch import nn
# --- TRAINING AND TESTING BLOCK ---
if __name__ == "__main__":
    np.random.seed(42)
    random.seed(42)

    # # -------- TRAIN PPO AGENT ----------
    env = AssemblyLineGymEnv()
    policy_kwargs = dict(activation_fn=nn.ReLU,)
    model = PPO(
        "MlpPolicy",
        env,
        verbose=1,
        policy_kwargs=policy_kwargs,
        tensorboard_log="./ppo_logs",
        n_steps=1024,     # Tune as needed for stability
        batch_size=256,   # Tune for performance
        learning_rate=linear_schedule
    )
    print("=== Save randomly initialized policy... ===")
    animate_validation(model, env, save_path="ppo_validation_rann.mp4", show=False)
    print("=== Training PPO agent... ===")
    model.learn(total_timesteps=50000)   # Increase for better convergence

    # --- Save trained model ---
    model.save("ppo_assemblyline.zip")

    # -------- TEST TRAINED AGENT ----------
    print("\n=== Testing trained agent ===")
    obs, _ = env.reset()
    done = False
    total_reward = 0
    step = 0
    while not done:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, _, _ = env.step(action)
        total_reward += reward
        if step % 10 == 0:
            print(f"Step {step}, reward: {reward:.2f}, platform: {obs[-1]*CONFIG['RAW_MATERIALS']:.1f}")
            env.render()
        step += 1






    animate_validation(model, env, save_path="ppo_validation.mp4", show=False)    model = PPO.load("ppo_assemblyline.zip", env)    # To reload later, use:    # -------- HOW TO RESUME/LOAD --------    print(f"Test episode total reward: {total_reward}")    # To reload later, use:
    model = PPO.load("ppo_assemblyline.zip", env)
    animate_validation(model, env, save_path="ppo_validation.mp4", show=False)

IndentationError: unexpected indent (3638612078.py, line 772)

In [ ]:
# Simple validation test
print("Testing basic functionality...")

# Test CONFIG is available
print(f"CONFIG loaded: {CONFIG['RAW_MATERIALS']} bottles target")

# Test creating a simple environment
env = simpy.Environment()
buffer = Buffer(env, 'test', 10)
print(f"Buffer created: {buffer.name} with capacity {buffer.capacity}")

print("✓ Basic validation passed!")
print("The improved classes and CONFIG are working correctly.")

# Show CONFIG structure
print("\n=== CONFIG Structure ===")
for key, value in CONFIG.items():
    if isinstance(value, dict):
        print(f"{key}: {list(value.keys())}")
    else:
        print(f"{key}: {value}")